# **Boruta Shap Feature Analysis**
**what I want to learn:**
- **which features (taxonomic families) are important to blooms at day 3?**

**k fold cross validation documentation:**
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

**Needed libraries**

In [1]:
import pandas as pd
import numpy as np
from BorutaShap import BorutaShap
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier)

**Functions**

In [2]:
## generate x and y dataframes for machine learning and/or boruta shap/kfold cross validation
def make_xy_tables(meta_df,
                   otu_df,
                   merge_on,
                   y_col):
    output_dict = {}
    
    mini_meta = meta_df.loc[:, (merge_on, y_col)]
    comb_df = otu_df.merge(mini_meta, how="left", on=[merge_on])
    ## x - the side that has the data I want the model to use to predict y
    pre_x_df = comb_df.copy()
    x_df = pre_x_df.drop(y_col, axis=1)
    x_df[merge_on] = x_df[merge_on].astype(float)
    x_df = x_df.drop(merge_on, axis=1)
    ## y - what is to be predicted
    y_df = comb_df[y_col]

    ## saving my outputs
    output_dict.update({"x_dataframe": x_df,
                        "y_dataframe": y_df})
    return(output_dict)


## run kfold cross validation on x and y datasets and then identify important features via boruta shap
def kfold_boruta_shap(k_fold,
                      feature_selector,
                      x_dataframe,
                      y_dataframe,
                      trial_num,
                      run_name):
    output_dict = {}

    train_list = []
    test_list = []
    for i, (train_index, test_index) in enumerate(k_fold.split(x_dataframe, y_dataframe)):
        print(f"Fold {i}:")
        print(f"Training dataset index: {train_index}")
        print(f"Testing dataset index: {test_index}")
        ## setting up test/train datasets 
        x_train = x_dataframe.filter(items=train_index, axis=0)
        x_test = x_dataframe.filter(items=test_index, axis=0)
        y_train = y_dataframe.filter(items=train_index, axis=0)
        y_test = y_dataframe.filter(items=test_index, axis=0)

        ## running boruta shap on training data
        feature_selector.fit(X=x_train,
                             y=y_train, 
                             n_trials=trial_num,
                             random_state=0,
                             sample=False,
                             verbose=True)
        
        train_acc_features = feature_selector.accepted
        train_list = train_list + train_acc_features
        
        ## running borta shap on testing data
        feature_selector.fit(X=x_test,
                             y=y_test, 
                             n_trials=trial_num,
                             random_state=0,
                             sample=False,
                             verbose=True)
        
        test_acc_features = feature_selector.accepted
        test_list = test_list + test_acc_features

    ## saving my outputs
    output_dict.update({f"{run_name}_train": train_list,
                        f"{run_name}_test": test_list})
    return(output_dict)


## to make the output table of the kfold cross validated boruta shap results
def create_occurence_table(input_list):
    wanted_df = pd.DataFrame(np.unique(input_list, return_counts=True)).T
    wanted_df.columns = ["feature", "num_occurences"]
    wanted_df = wanted_df.sort_values(by="num_occurences", ascending=False)
    wanted_df["av_occurences"] = wanted_df["num_occurences"]/5
    return(wanted_df) 

**File paths**

In [4]:
family_deltas_fp = "../data/family_deltas_wide.tsv"
meta_fp = "../data/ml_approved_metadata.tsv"
meta_keys_fp = "../data/meta_dict_keys.tsv"

## common variables for functions
bs_cols = ['family', 'num_occurences']
bs_sort_by = 'num_occurences'

## cross validator and boruta shap 
kf = KFold(n_splits=5)
grad_boost = GradientBoostingClassifier(n_estimators=100)
random_forest_bs = BorutaShap(importance_measure='shap', 
                           classification=False)
grad_boost_bs = BorutaShap(model=grad_boost,
                        importance_measure='shap',
                        classification=True)
borutaShap_dict = {"random_forest": random_forest_bs}

**Reading in files and data wrangling**

In [5]:
family_deltas_df = pd.read_csv(family_deltas_fp, sep='\t')
meta = pd.read_csv(meta_fp, sep='\t')
meta_keys = pd.read_csv(meta_keys_fp, sep='\t')

In [5]:
## pulling out mouse id assigned numbers as a dictionary to map it to the deltas table
mouse_id_dict = dict(zip(meta_keys.mouse_id, meta_keys.assigned_num))
inverse_mouseID_dict = dict(zip(meta_keys.assigned_num, meta_keys.mouse_id))

In [9]:
## converting mouse ids in the deltas table to the same numbers as the metadata
family_deltas_df["mouse_id"] = family_deltas_df["mouse_id"].map(mouse_id_dict)

## Bloom Status

In [6]:
## more data wrangling bc why not i love it!!! (that's sarcasm)
## im actually just generating my x and y dataframes 
bloom_xy = make_xy_tables(meta_df=meta,
                          otu_df=family_deltas_df,
                          merge_on="mouse_id",
                          y_col="bloom_status")

bloom_x_df = bloom_xy["x_dataframe"]
bloom_y_df = bloom_xy["y_dataframe"]

In [8]:
bloom_bs_acc_train = {}
bloom_bs_acc_test = {}
for label, boruta_shap in borutaShap_dict.items():
    bloom_results = kfold_boruta_shap(k_fold=kf,
                                      feature_selector=boruta_shap,
                                      x_dataframe=bloom_x_df,
                                      y_dataframe=bloom_y_df,
                                      trial_num=100,
                                      run_name="bloom")
    
    ## pulling out boruta shap accepted features 
    ## training data
    bloom_train_list = bloom_results["bloom_train"]
    bs_bloom_train_df = create_occurence_table(input_list=bloom_train_list)
    bs_bloom_train_df["bs_model"] = label

    ## testing data
    bloom_test_list = bloom_results["bloom_test"]
    bs_bloom_test_df = create_occurence_table(input_list=bloom_test_list)
    bs_bloom_test_df["bs_model"] = label

    bloom_bs_acc_train.update({f"{label}_accepted": bs_bloom_train_df})
    bloom_bs_acc_test.update({f"{label}_accepted": bs_bloom_test_df})


bloom_train_features = pd.concat(bloom_bs_acc_train, ignore_index=True)
bloom_test_features = pd.concat(bloom_bs_acc_test, ignore_index=True)

Fold 0:
Training dataset index: [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49]
Testing dataset index: [0 1 2 3 4 5 6 7 8 9]


  0%|          | 0/100 [00:00<?, ?it/s]

5 attributes confirmed important: [' f__Akkermansiaceae', ' f__Peptostreptococcaceae', ' f__Erysipelotrichaceae', ' f__[Eubacterium]_coprostanoligenes_group', ' f__Enterobacteriaceae']
46 attributes confirmed unimportant: [' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Sutterellaceae', ' f__Erysipelatoclostridiaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Rikenellaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Rumin

  0%|          | 0/100 [00:00<?, ?it/s]

1 attributes confirmed important: [' f__Enterococcaceae']
51 attributes confirmed unimportant: [' f__Peptostreptococcaceae', ' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Sutterellaceae', ' f__Enterobacteriaceae', ' f__Erysipelatoclostridiaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Erysipelotrichaceae', ' f__Rikenellaceae', ' f__Sphingomonadaceae', ' f__UCG-010', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonadaceae', ' f__Akkermansia

...

2 attributes confirmed important: [' f__Akkermansiaceae', ' f__Enterobacteriaceae']
50 attributes confirmed unimportant: [' f__Peptostreptococcaceae', ' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Sutterellaceae', ' f__Erysipelatoclostridiaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Erysipelotrichaceae', ' f__Rikenellaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__UCG-010', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonada

  0%|          | 0/100 [00:00<?, ?it/s]

1 attributes confirmed important: [' f__Akkermansiaceae']
51 attributes confirmed unimportant: [' f__Peptostreptococcaceae', ' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Sutterellaceae', ' f__Enterobacteriaceae', ' f__Erysipelatoclostridiaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Erysipelotrichaceae', ' f__Rikenellaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__UCG-010', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonada

  0%|          | 0/100 [00:00<?, ?it/s]

4 attributes confirmed important: [' f__UCG-010', ' f__Peptostreptococcaceae', ' f__Sutterellaceae', ' f__Rikenellaceae']
48 attributes confirmed unimportant: [' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Enterobacteriaceae', ' f__Erysipelatoclostridiaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Erysipelotrichaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonadaceae', ' f__Akkermansia

- f_RF39: unclassified enterococcaceae bacterium
- f_UCG-010: i have no idea, after looking it up on the silva database website (the database I used to classify the taxonomy for this data in qiime), they're not sure where to place it (insertae sedis) but there's a bunch of uncultured clostridiales/clostridiaceae bacterium under it (I wouldn't be too dependent on that though bc there's 973 unclassified bacterium that are under UCG-010)

In [9]:
bloom_train_features

,feature,num_occurences,av_occurences,bs_model
0,f__Akkermansiaceae,5,1.0,random_forest
1,f__Peptostreptococcaceae,5,1.0,random_forest
2,f__Erysipelotrichaceae,4,0.8,random_forest
3,f__Enterobacteriaceae,3,0.6,random_forest
4,f__[Eubacterium]_coprostanoligenes_group,3,0.6,random_forest
5,f__Clostridia_UCG-014,2,0.4,random_forest
6,f__RF39,2,0.4,random_forest
7,f__UCG-010,2,0.4,random_forest
8,f__Enterococcaceae,1,0.2,random_forest
9,f__Tannerellaceae,1,0.2,random_forest


In [10]:
bloom_test_features

,feature,num_occurences,av_occurences,bs_model
0,f__Enterococcaceae,2,0.4,random_forest
1,f__Peptostreptococcaceae,2,0.4,random_forest
2,f__Eggerthellaceae,1,0.2,random_forest
3,f__Enterobacteriaceae,1,0.2,random_forest
4,f__Erysipelotrichaceae,1,0.2,random_forest
5,f__Lactobacillaceae,1,0.2,random_forest
6,f__Rikenellaceae,1,0.2,random_forest
7,f__Ruminococcaceae,1,0.2,random_forest
8,f__Sutterellaceae,1,0.2,random_forest
9,f__Tannerellaceae,1,0.2,random_forest


## Diet

In [11]:
## creating x and y dataframes for kfold and boruta shap analysis
diet_xy = make_xy_tables(meta_df=meta,
                         otu_df=family_deltas_df,
                         merge_on="mouse_id",
                         y_col="diet")

diet_x_df = diet_xy["x_dataframe"]
diet_y_df = diet_xy["y_dataframe"]

In [12]:
## I can't use both random forest and gradient boosting classifiers on the diet data since it's not binary
## and its not supported :(
diet_results = kfold_boruta_shap(k_fold=kf,
                                 feature_selector=random_forest_bs,
                                 x_dataframe=diet_x_df,
                                 y_dataframe=diet_y_df,
                                 trial_num=100,
                                 run_name='diet')

Fold 0:
Training dataset index: [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49]
Testing dataset index: [0 1 2 3 4 5 6 7 8 9]


  0%|          | 0/100 [00:00<?, ?it/s]

3 attributes confirmed important: [' f__Peptostreptococcaceae', ' f__Muribaculaceae', ' f__Oscillospiraceae']
45 attributes confirmed unimportant: [' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Sutterellaceae', ' f__Enterobacteriaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Moorellaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Erysipelotrichaceae', ' f__Rikenellaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__UCG-010', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonadaceae', ' f__Eggerthellaceae', ' f__Akkermansiaceae', ' f__AKAU3644', ' f__Clostridiaceae', ' f__Clostridi

  0%|          | 0/100 [00:00<?, ?it/s]

3 attributes confirmed important: [' f__Lactobacillaceae', ' f__Erysipelatoclostridiaceae', ' f__Anaerovoracaceae']
49 attributes confirmed unimportant: [' f__Peptostreptococcaceae', ' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Sutterellaceae', ' f__Enterobacteriaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Erysipelotrichaceae', ' f__Rikenellaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__UCG-010', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonadaceae', ' f__Akkermansia

...

5 attributes confirmed important: [' f__Akkermansiaceae', ' f__Erysipelotrichaceae', ' f__Streptococcaceae', ' f__Sutterellaceae', ' f__Tannerellaceae']
47 attributes confirmed unimportant: [' f__Peptostreptococcaceae', ' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Enterobacteriaceae', ' f__Erysipelatoclostridiaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Rikenellaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__UCG-010', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonada

  0%|          | 0/100 [00:00<?, ?it/s]

5 attributes confirmed important: [' f__Eggerthellaceae', ' f__Peptostreptococcaceae', ' f__Sutterellaceae', ' f__Erysipelatoclostridiaceae', ' f__Tannerellaceae']
46 attributes confirmed unimportant: [' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Enterobacteriaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Erysipelotrichaceae', ' f__Rikenellaceae', ' f__Sphingomonadaceae', ' f__UCG-010', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonadaceae', ' f__Akkermansia

  0%|          | 0/100 [00:00<?, ?it/s]

4 attributes confirmed important: [' f__Peptostreptococcaceae', ' f__UCG-010', ' f__Sutterellaceae', ' f__Rikenellaceae']
48 attributes confirmed unimportant: [' f__Prevotellaceae', ' f__Deferribacteraceae', ' f__Acholeplasmataceae', ' f__Oscillospiraceae', ' f__Enterobacteriaceae', ' f__Erysipelatoclostridiaceae', ' f__Lachnospiraceae', ' f__Moraxellaceae', ' f__Anaerovoracaceae', ' f__Desulfovibrionaceae', ' f__Saccharimonadaceae', ' f__Bifidobacteriaceae', ' f__Bacillaceae', ' f__Muribaculaceae', ' f__Moorellaceae', ' f__Peptococcaceae', ' f__Streptococcaceae', ' f__Hungateiclostridiaceae', ' f__Morganellaceae', ' f__Oscillospirales', ' f__Weeksellaceae', ' f__uncultured', ' f__Monoglobaceae', ' f__Lactobacillaceae', ' f__Erysipelotrichaceae', ' f__Enterococcaceae', ' f__Sphingomonadaceae', ' f__Clostridia_UCG-014', ' f__Christensenellaceae', ' f__Tannerellaceae', ' f__Anaerofustaceae', ' f__Marinifilaceae', ' f__RF39', ' f__Ruminococcaceae', ' f__Xanthomonadaceae', ' f__Akkermansia

In [13]:
## selected features from boruta shap on training data 
diet_train_list = diet_results["diet_train"]
bs_diet_train_df = create_occurence_table(input_list=diet_train_list)

bs_diet_train_df

,feature,num_occurences,av_occurences
0,f__Eggerthellaceae,3,0.6
2,f__Enterococcaceae,3,0.6
1,f__Enterobacteriaceae,2,0.4
3,f__Erysipelatoclostridiaceae,2,0.4
7,f__Peptostreptococcaceae,2,0.4
8,f__Sutterellaceae,2,0.4
4,f__Erysipelotrichaceae,1,0.2
5,f__Muribaculaceae,1,0.2
6,f__Oscillospiraceae,1,0.2
9,f__Tannerellaceae,1,0.2


In [14]:
## selected features from boruta shap on testing data
diet_test_list = diet_results["diet_test"]
bs_diet_test_df = create_occurence_table(input_list=diet_test_list)

bs_diet_test_df

,feature,num_occurences,av_occurences
12,f__Tannerellaceae,3,0.6
0,f__Akkermansiaceae,2,0.4
3,f__Enterococcaceae,2,0.4
4,f__Erysipelatoclostridiaceae,2,0.4
5,f__Erysipelotrichaceae,2,0.4
6,f__Lactobacillaceae,2,0.4
8,f__Peptostreptococcaceae,2,0.4
11,f__Sutterellaceae,2,0.4
1,f__Anaerovoracaceae,1,0.2
2,f__Enterobacteriaceae,1,0.2


**Saving my outputs**

In [16]:
## bloom status
bloom_train_features.to_csv("../data/outputs/borutaShap_bloom_train.tsv", sep="\t")
bloom_test_features.to_csv("../data/outputs/borutaShap_bloom_test.tsv", sep="\t")

## diet
bs_diet_train_df.to_csv("../data/outputs/borutaShap_diet_train.tsv", sep="\t")
bs_diet_test_df.to_csv("../data/outputs/borutaShap_diet_test.tsv", sep="\t")